In [3]:
#===============
#   IMPORTS
#===============

# Fundamentals
import numpy as np
import pandas as pd

# Dash
import dash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input

# Dash in JupyterLab
from jupyter_dash import JupyterDash

# Plotly
import plotly.graph_objects as go
import plotly.io as pio

from plotly.tools import mpl_to_plotly

pio.renderers.default = 'iframe'


#-STYLES----------------------------------
linestyles = ['-', '--', '-.', ':', (0, (5, 2, 1, 2, 1, 2))]
markers = ['o', 'x', 's', '^', 'd']
colors = ['#0077BB', '#33BBEE', '#009988', '#EE7733', '#CC3311', '#EE3377', '#BBBBBB']
colors4 = ['#0077BB', '#EE7733', '#CC3311', '#EE3377']
blues = ['#eff3ff','#c6dbef','#9ecae1','#6baed6','#3182bd','#08519c']
greys5 = ['#f7f7f7','#cccccc','#969696','#636363','#252525']
greys6 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']
greys7 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys8 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys9 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525','#000000']
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']


#====================
#   PY INCLUSIONS
#====================

# Data import
import baptistery_data_input as bdi
prism_data, levelling_data, extensimeter_data = bdi.readAllData()
prism_pos, levelling_pos, extensimeter_pos, positions = bdi.readSensorPositions()

def reformatPlot(fig, size=None, secondary=False):
    if size != None:
        fig.update_layout(height=size[1], width=size[0])
        
    fig.update_layout(xaxis_showline=True,
                      xaxis_mirror='ticks',
                      xaxis_linewidth=2,
                     yaxis_showline=True,
                     yaxis_mirror='ticks',
                     yaxis_linewidth=2)
    fig.update_layout(
        xaxis_ticks="inside",
        xaxis_ticklen=8,
        xaxis_tickwidth=2,
        yaxis_ticks="inside",
        yaxis_ticklen=8,
        yaxis_tickwidth=2
    )
    fig.update_layout(
        font_family="Roboto",
        hoverlabel_font_family="Roboto",
        xaxis_title_font_size=16,
        xaxis_tickfont_size=16,
        xaxis_showgrid=True,
        yaxis_title_font_size=16,
        yaxis_tickfont_size=16,
        yaxis_showgrid=True,
    )
    
    if secondary:    
        fig.update_layout(
                yaxis2=dict(
                title_font_size=16,
                tickfont_size=16,
                showgrid=False,
                showline=True,
                linewidth=2,
                ticks='inside',
                ticklen=8,
                tickwidth=2,
                title_font_color='gray',
                tickcolor='lightgray',
                tickfont_color='gray'
            ),
            yaxis_mirror=False
        )

    return fig

def selectPrismSection(n, full=False):
    """
    NOTE: for now, the function excludes prisms in the 
    1xx series.    
    """
    full_section = {
        '01':['01','07'],
        '02':['02','08'],
        '03':['03','09'],
        '04':['04','10'],
        '05':['05','11'],
        '06':['06','12'],
        '07':['07','01'],
        '08':['08','02'],
        '09':['09','03'],
        '10':['10','04'],
        '11':['11','05'],
        '12':['12','06']
    }
    if full:
        n = full_section[n]
    else:
        n = [n]
    
    selected_sensors = [p for p in prism_pos.index if p.endswith(n[0]) and not (p.startswith('1'))]
    if full:
        selected_sensors += [p for p in prism_pos.index if p.endswith(n[1]) and not (p.startswith('1'))]
    
    return selected_sensors    

def rotTraslPrism(df, date):
    """
    Returns east and z coordinates of prisms
    in the DataFrame *df* for the date *date*.
    """
    x = df.loc[date, (slice(None), 'x')].values
    y = df.loc[date, (slice(None), 'y')].values
    z = df.loc[date, (slice(None), 'z')].values
    
    # Traslation
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    x1 = x - ref_x
    y1 = y - ref_y
    
    # Rotation
    a = -np.arctan(y1/x1)
    east = x1*np.cos(a) - y1*np.sin(a)
    north = x1*np.sin(a) + y1*np.cos(a)

    if (north > 10**(-6)).any():
        print("ERROR: north coordinate is not zero")
        return 1
    
    return east, z

In [4]:
def xyToEN(x, y):
    """
    Converts (x, y) coordinates of prisms to (E, N) ones.
    It takes either single or array-like coordinates.
    """
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    rot = np.deg2rad(37.1)
    e = x - ref_x
    n = y - ref_y
    eR = e*np.cos(rot) - n*np.sin(rot)
    nR = e*np.sin(rot) + n*np.cos(rot)
   
    return eR, nR

In [5]:
from plotly.subplots import make_subplots

In [6]:
def figurePrismDisplacement(p):
    """
    Produces a figure with the displacement of the *p*
    prism. Both the total one and single components.
    """
    
    fig = go.Figure(layout_template=None)
    fig.update_layout(
        margin = dict(t=40, b=40),
    )
    
    fig = make_subplots(specs=[[{"secondary_y": True}]],
                        shared_xaxes=True,
                        vertical_spacing=0.0,
                        figure=fig)
    fig = reformatPlot(fig, size=[1200, 350], secondary=True)
    
    x = prism_data[p, 'x'].values
    y = prism_data[p, 'y'].values
    z = prism_data[p, 'z'].values*1000.
    e, n = xyToEN(x, y)
    e = e*1000.
    n = n*1000.

    de = e - e[0]
    dn = n - n[0]
    dz = z - z[0]

    dtot = np.sqrt(de**2 + dn**2 + dz**2)

    r = np.sqrt(e**2 + n**2)
    dr = r - r[0]

    alpha = np.arctan(n/e)
    dalpha = alpha - alpha[0]
    dtan = r*np.sin(dalpha)

    traces = [dtot, dr, dtan, dz]
    names = ['Total', 'Radial', 'Tangential', 'Vertical']

    for t,n,c in zip(traces, names, colors4):
        fig.add_trace(
            go.Scatter(
                x=prism_data.index,
                y=t,
                mode='markers+lines',
                name=n,
                marker_color = c,
                line_color = c,
            ),
            secondary_y = False,
        )

    fig.add_trace(
        go.Scatter(
            x=extensimeter_data.index,
            y=extensimeter_data['F4F8', 'temp'].rolling(24).mean(),
            line_dash='dot',
            line_color='gray',
            name='Temperature'
        ),
        secondary_y = True,
    )

    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.02,
        y=0.95,
        text='<b>'+p+'</b>',
        font_family='Roboto',
        font_color='black',
        font_size=14,
        borderpad=4,
        bordercolor='black',
        borderwidth=1.5,
        showarrow=False
    )

    fig.update_yaxes(title_text="Displacement [mm]", secondary_y=False)
    fig.update_yaxes(title_text="Temperature [°C]", secondary_y=True)
            
    return fig

In [7]:
def reformatPlot(fig, size=None, secondary=False):
    """
    Updates the layout of all subplots.
    """
    if size != None:
        fig.update_layout(height=size[1], width=size[0])
        
    fig.update_xaxes(
        showline = True,
        mirror = 'ticks',
        linewidth = 2,
        ticks = 'inside',
        ticklen = 8,
        tickwidth = 2,
        showgrid = True,
        title_font_size=16,
        tickfont_size=16,
    )
    fig.update_yaxes(
        showline = True,
        mirror = 'ticks',
        linewidth = 2,
        ticks = 'inside',
        ticklen = 8,
        tickwidth = 2,
        showgrid = True,
        title_font_size=16,
        tickfont_size=16,
    )
    
    if secondary:
        fig.update_layout(
                yaxis2=dict(
                title_font_size=16,
                tickfont_size=16,
                showgrid=False,
                showline=True,
                linewidth=2,
                ticks='inside',
                ticklen=8,
                tickwidth=2,
                title_font_color='gray',
                tickcolor='lightgray',
                tickfont_color='gray'
            ),
            yaxis_mirror=False
        )
        
    return fig 

In [8]:
figurePrismDisplacement('406')

In [89]:
def figurePrismSelection():
    """
    A figure from which to select prisms.
    """
    fig = go.Figure(layout_template='plotly_white')
    
    fig = make_subplots(rows=1, cols=3, 
                        subplot_titles=('Ground floor', 'First floor', 'Second floor'),
                        figure=fig
                       )
    fig.update_layout(height=350, width=1000,
                         margin=dict(l=0,r=0,b=0,t=50))
    
    ground_floor = ['101', '102', '103', '104', 'P1']
    first_floor = [p for p in prism_pos.index if (p[0] == '2' or p[0] == '3')]
    second_floor = [p for p in prism_pos.index if (p[0] == '4' or p[0] == '5')]
    
    floors = [ground_floor, first_floor, second_floor]
    
    for i, f in enumerate(floors):
        x = prism_pos.loc[f]['radius']*np.cos(np.deg2rad(prism_pos.loc[f]['angle']))
        y = prism_pos.loc[f]['radius']*np.sin(np.deg2rad(prism_pos.loc[f]['angle']))
        # Add the shape of the Baptistery
        fig.add_shape(type="circle",
            xref="x", yref="y",
            x0=-17.80, y0=-17.80, x1=17.80, y1=17.80,
            line_color="lightgrey",
            row=1, col=i+1
        )                
        fig.add_shape(type="circle",
            xref="x", yref="y",
            x0=-15.25, y0=-15.25, x1=15.25, y1=15.25,
            line_color="lightgrey",
            line_width=1,
            row=1, col=i+1
        )
        # Add the prisms
        fig.add_trace(
            go.Scatter(
                x=x, y=y,
                mode='markers',
                marker_color=colors[1],
                hovertext=['Prism n. {}'.format(p) 
                             for p in f],
                marker_size=10,
                hoverinfo='text',
                selected_marker_color='red'
              ),
            row=1, col=i+1
        )
        
    # Disable the legend
    fig.update(layout_showlegend=False)
    
    # Format plot
    fig.update_xaxes(
        range = (-18, 18),
        zeroline = False,
        showgrid = False,
        visible = False,
    )
    fig.update_yaxes(
        range = (-18, 18),
        zeroline = False,
        showgrid = False,
        visible = False,
    )
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
    )
    
    fig.update_layout(clickmode='event+select')
    
    return fig

In [90]:
figurePrismSelection()